In [1]:

import os
import yaml
import pandas as pd

ROOT_DIR = "/home/melodino/veh_type_pred"

In [2]:
def load_yaml(file_path):
    """Load a YAML file and return the content."""
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

def parse_dependencies(dependencies):
    """Parse the list of dependencies into a dictionary of package, version, and build."""
    dep_dict = {}
    for dep in dependencies:
        if isinstance(dep, str):
            parts = dep.split('=')
            if len(parts) == 3:
                package, version, build = parts
                dep_dict[package] = {'version': version, 'build': build}
            elif len(parts) == 2:
                package, version = parts
                dep_dict[package] = {'version': version, 'build': None}
            else:
                dep_dict[package] = {'version': None, 'build': None}  # No version or build specified
        elif isinstance(dep, dict):
            pip_packages = dep['pip']
            for pip_pkg in pip_packages:
                package, version = pip_pkg.split('==')
                dep_dict[package] = {'version': version, 'build': 'pip'}
    return dep_dict


def compare_envs(env1, env2):
    """Compare the dependencies of two environments."""
    dep1 = parse_dependencies(env1.get('dependencies', []))
    dep2 = parse_dependencies(env2.get('dependencies', []))
    
    all_keys = set(dep1.keys()).union(set(dep2.keys()))
    data = []

    for key in all_keys:
        env1_info = dep1.get(key, {'version': 'non_existent', 'build': 'non_existent'})
        env2_info = dep2.get(key, {'version': 'non_existent', 'build': 'non_existent'})

        # Add to data only if builds are different or the package exists in only one environment
        if env1_info['build'] != env2_info['build'] or env1_info['version'] == 'non_existent' or env2_info['version'] == 'non_existent':
            data.append([
                key,
                env1_info['version'],
                env1_info['build'],
                env2_info['version'],
                env2_info['build']
            ])
    
    # Create a DataFrame from the collected data
    df = pd.DataFrame(data, columns=['Package', 'Env1_Version', 'Env1_Build', 'Env2_Version', 'Env2_Build'])
    return df

In [4]:
# Load the environments
env1 = load_yaml(f"{ROOT_DIR}/env_yml_files/base_2023_09_env.yml")
env2 = load_yaml(f"{ROOT_DIR}/env_yml_files/tslearn_and_base_2023_09_env.yml")

# Compare environments and generate DataFrame
comparison_df = compare_envs(env1, env2)
comparison_df.sort_values(by=['Package'], inplace=True)

# Print the DataFrame
with pd.option_context('display.max_rows', None):  # Sets max_rows to None within the block
    display(comparison_df)

,Package,Env1_Version,Env1_Build,Env2_Version,Env2_Build
3,ca-certificates,2023.08.22,haa95532_0,2024.2.2,h56e8100_0
4,certifi,2023.7.22,py311haa95532_0,2024.2.2,pyhd8ed1ab_0
7,openssl,3.0.10,h2bbff1b_2,3.2.1,hcfcfb64_1
6,python_abi,non_existent,non_existent,3.11,2_cp311
0,tslearn,non_existent,non_existent,0.6.3,py311h59ca53f_1
5,ucrt,non_existent,non_existent,10.0.22621.0,h57928b3_0
1,vc14_runtime,non_existent,non_existent,14.38.33130,h82b7239_18
2,vs2015_runtime,14.27.29016,h5e58377_2,14.38.33130,hcb4865c_18


In [1]:
# # When I try to conda install -c conda-forge tslearn on the base environment for 2023_09 anaconda with no other additional packages (which is what is installed on the gaming laptop)
# The following packages will be downloaded:

#     package                    |            build
#     ---------------------------|-----------------
#     ca-certificates-2024.2.2   |       h56e8100_0         152 KB  conda-forge
#     certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
#     openssl-3.2.1              |       hcfcfb64_1         7.8 MB  conda-forge
#     python_abi-3.11            |          2_cp311           5 KB  conda-forge
#     tslearn-0.6.3              |  py311h59ca53f_1         450 KB  conda-forge
#     ucrt-10.0.22621.0          |       h57928b3_0         1.2 MB  conda-forge
#     vc14_runtime-14.38.33130   |      h82b7239_18         732 KB  conda-forge
#     vs2015_runtime-14.38.33130 |      hcb4865c_18          17 KB  conda-forge
#     ------------------------------------------------------------
#                                            Total:        10.6 MB

# The following NEW packages will be INSTALLED:

#   python_abi         conda-forge/win-64::python_abi-3.11-2_cp311
#   tslearn            conda-forge/win-64::tslearn-0.6.3-py311h59ca53f_1
#   ucrt               conda-forge/win-64::ucrt-10.0.22621.0-h57928b3_0
#   vc14_runtime       conda-forge/win-64::vc14_runtime-14.38.33130-h82b7239_18

# The following packages will be UPDATED:

#   ca-certificates    pkgs/main::ca-certificates-2023.08.22~ --> conda-forge::ca-certificates-2024.2.2-h56e8100_0
#   certifi            pkgs/main/win-64::certifi-2023.7.22-p~ --> conda-forge/noarch::certifi-2024.2.2-pyhd8ed1ab_0
#   openssl              pkgs/main::openssl-3.0.10-h2bbff1b_2 --> conda-forge::openssl-3.2.1-hcfcfb64_1
#   vs2015_runtime     pkgs/main::vs2015_runtime-14.27.29016~ --> conda-forge::vs2015_runtime-14.38.33130-hcb4865c_18